# $VaR$ de Instrumentos de Deuda

#### por Carlos Santillán

## Método de Variación Total

Para valuar, nos interesan:

- Fecha de emisión 

- Fecha de vencimiento

- VN

- Tasa de interés

- Periodicidad de la tasa de Interés

- Tasa de rendimiento del mercado (YTM)

Nuestro objetivo ahora será determinar el $VaR$ diario con un nivel de confianza del 99.9%
para el Bono de Lala

En este caso:

- Fecha de emisión: 12 marzo 2018 

- Fecha de vencimiento: 28 febrero 2028

- VN: 100

- Tasa de interés: 9.17%

- Periodicidad de la tasa de Interés: 182 días



### Bloque Inicial

In [7]:
import numpy as np                  # Vectores
import pandas as pd                 # DataFrames
import matplotlib.pyplot as plt     # Gráficas
import math                         # Operaciones
import seaborn as sns               # Gráficos y controles de Scrapping
import random                       # Muestreos aleatorios
import quandl                       # Datasets financieros
import yahoo_finance                # Acceso a Yahoo Finanzas
import selenium
import time
import pyautogui                    # Controla el cursor
import PIL                          # Para manipular imágenes

### Tools for Monte Carlo Simulation
from IPython.display import clear_output

### Tools for dates
from datetime import datetime
from datetime import date

### selenium package tools
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC1
from selenium.webdriver.common.by import By

### SciPy Dstributions
from scipy import stats             # Paquetería estadística
from scipy.stats import kstest      # Prueba de Kolmogorov-Smirnov
from scipy.stats import beta        # Distribución Beta
from scipy.stats import expon       # Distribución Exponencial
from scipy.stats import uniform     # Distribución uniforme
from scipy.stats import norm        # Distribución normal
from scipy.stats import binom       # Distribución Binomial

### PIL image handler
from PIL import Image

### Forced installation of pandas_datareader
import pip
from pip._internal import main
main(['install', 'pandas_datareader'])
#main(['install', 'fix_yahoo_finance'])
from pandas_datareader import data as web
#import fix_yahoo_finance


%matplotlib inline

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Por la periodicidad del bono, hay un tramo que ya venció y lo queremos valuar
al día de hoy.

- Días por vencer será: HOY() - fecha de vencimiento

- Número de flujos: dias por vencer / plazo cupón

- Valor cupón: VN * tasa de 182 días * plazo/360

- Número cupones completos: entero(numero de flujos) (son 16)

- Días por devengar: dias por vencer - (182 * numero cupones completos)

- Días devengados: plazo - dias por devengar

(DONE) Habrá que traer los flujos a *t + 1*  

precio sucio es 100.16

precio limpio (sólo sirve para cotización) es: 

(DONE) Obtener variaciones (primero dividir entre 100) 

(DONE) Obtener variación al cuadrado 

Obtener media y desviación estándar

Aplicar VaR a las variaciones de las tasas

Tasa frontera = Tasa de ercado*(1 + Var)

Volvemos a obtener precio sucio con Tasa Frontera

Obtenemos un diferencial (-0.7728) 

plazo ceros = plazo + 143

plazo ceros / 360

% ponderacion = flujos hoy / precio sucio = 0.00446

Dduracion de Macualay = suma (% Ponderacion * plazo ceros / 360) = 6.0097

Duración modificada = -5.7395 (menos la anterior traida un periodo a valor presente)

Convexidad = 46.1374

Obtener VaR con Duracion Modificada y Convexidad


Leemos los datos de *LALA 18*:

In [2]:
datos_LALA18 = pd.read_csv('C:/Users/PC/Desktop/ULSA/7 Semestre/Admon de Riesgos/LALA18.csv')
datos_LALA18.head()

,Date,YTM
0,17/10/2018,9.488
1,18/10/2018,9.459
2,19/10/2018,9.604
3,22/10/2018,9.769
4,23/10/2018,9.775


In [3]:
datos_LALA18.tail()

,Date,YTM
246,11/10/2019,9.364
247,14/10/2019,9.374
248,15/10/2019,9.363
249,16/10/2019,9.308
250,17/10/2019,9.311


Vemos nuestros tipos de datos:

In [4]:
datos_LALA18.dtypes

Date     object
YTM     float64
dtype: object

Declaramos la información existente:

In [5]:
fechaEmision = pd.to_datetime('2018-03-12')
fechaVencimiento = pd.to_datetime('2028-02-28')
VN = 100
tasaInteres = 0.0917
periodicidad = 182

Funciones para obtener los datos principales:

In [18]:
### INICIA FUNCIÓN ###
def dias_por_vencer(fecha):
    hoy = pd.to_datetime(date.today())
    resp = (-1)*(hoy - fecha).days
    return resp
### FIN DE FUNCIÓN ###

### INIVIA FUNCION ###
def numero_flujos(plazo):
    resp = dias_por_vencer(fechaVencimiento) / plazo
    return resp
### FIN DE FUNCIÓN ### 

### INICIA FUNCIÓN ###
def valor_cupon(valorNorminal, tasa, plazo):
    resp = valorNorminal * tasa * (plazo/360)
    return resp
### FIN DE FUNCIÓN ###

### INICIA FUCION ###
def numero_cupones_completos(numeroFlujos):
    resp = int(numeroFlujos)
    return resp
### FIN DE FUNCIÓN ###

### INICIA FUNCIÓN ###
def dias_por_devengar(plazo):
    resp = dias_por_vencer(fechaVencimiento) - (plazo * numero_cupones_completos(numero_flujos(plazo)))
    return resp
### FIN DE FUNCIÓN ###

### INICIA FUNCIÓN ###
def dias_devengados(plazo):
    resp = plazo - dias_por_devengar(plazo)
    return resp
### FIN DE FUNCIÓN ###

39

Asignamos valores a las funciones para que sea más fácil manipular después:

In [19]:
diasPorVencer = dias_por_vencer(fechaVencimiento)
numeroFlujos = numero_flujos(periodicidad)
valorCupon = valor_cupon(VN, tasaInteres, periodicidad)
numeroCuponesCompletos = numero_cupones_completos(numeroFlujos)
diasPorDevengar = dias_por_devengar(periodicidad)
diasDevengados = dias_devengados(periodicidad)

print("Los dias por vencer son: " + str(diasPorVencer))
print("El número de flujos es: " + str(numeroFlujos))
print("El valor del cupón es: " + str(valorCupon))
print("El número de cupones completos es: " + str(numeroCuponesCompletos))
print("Los dias por devengar son: " + str(diasPorDevengar))
print("Los dias devengados son: " + str(diasDevengados))

Los dias por vencer son: 3055
El número de flujos es: 16.785714285714285
El valor del cupón es: 4.635944444444444
El número de cupones completos es: 16
Los dias por devengar son: 143
Los dias devengados son: 39


Creamos función que obtenga las tasas en decimales, las variaciones y las variaciones
al cuadrado:

In [27]:
### INICIA FUNCIÓN ###
def variaciones_tasas(datos):
    """
    Función para generar las columnas restantes del DataFrame original
    :param datos: pandas DataFrame con 2 columnas: [fechas | tasas]
    :return: pandas DataFrame con 5 columnas: [fechas | tasas | tasas decimal | variacion | variacion**2] 
    """
    datos['Tasas_decimal'] = datos.iloc[:,1] / 100     ### 2a columna entre 100
    
    tasasNumpy = np.asarray(datos['Tasas_decimal'])    ### Pasamos a numpy para manipular
    
    variacionesNumpy = np.zeros(len(tasasNumpy)-1)     ### Inicializamos vectores
    variacionesCuadNumpy = np.zeros(len(tasasNumpy)-1)
    for i in range(len(variacionesNumpy)):             ### Llenamos vectores
        variacionesNumpy[i] = np.log(tasasNumpy[i+1] / tasasNumpy[i])
        variacionesCuadNumpy[i] = variacionesNumpy[i]**2
        
    variacionesNumpyPd = pd.Series(variacionesNumpy)
    variacionesCuadNumpyPd = pd.Series(variacionesCuadNumpy)     ### Regresamos a Pandas
    
    datos['Variaciones_Tasa'] = variacionesNumpyPd
    datos['Variaciones_Cuad_Tasa'] = variacionesCuadNumpyPd     ### Añadimos columnas
    
    ### Recorremos un lugar nuestros datos nuevos
    datos['Variaciones_Tasa'] = datos['Variaciones_Tasa'].shift(+1)
    datos['Variaciones_Cuad_Tasa'] = datos['Variaciones_Cuad_Tasa'].shift(+1)
    
    return datos
### FIN DE FUNCIÓN ###

datos_LALA18_variaciones = variaciones_tasas(datos_LALA18)
    

Observamos el resultado de nuestra función:

In [28]:
datos_LALA18_variaciones.head()

,Date,YTM,Tasas_decimal,Variaciones_Tasa,Variaciones_Cuad_Tasa
0,17/10/2018,9.488,0.09488,NaN,NaN
1,18/10/2018,9.459,0.09459,-0.003061,9.370780e-06
2,19/10/2018,9.604,0.09604,0.015213,2.314356e-04
3,22/10/2018,9.769,0.09769,0.017034,2.901717e-04
4,23/10/2018,9.775,0.09775,0.000614,3.769950e-07


In [29]:
datos_LALA18_variaciones.tail()

,Date,YTM,Tasas_decimal,Variaciones_Tasa,Variaciones_Cuad_Tasa
246,11/10/2019,9.364,0.09364,-0.002134,4.552086e-06
247,14/10/2019,9.374,0.09374,0.001067,1.139236e-06
248,15/10/2019,9.363,0.09363,-0.001174,1.378622e-06
249,16/10/2019,9.308,0.09308,-0.005892,3.470985e-05
250,17/10/2019,9.311,0.09311,0.000322,1.038460e-07


In [30]:
datos_LALA18_variaciones.dtypes

Date                      object
YTM                      float64
Tasas_decimal            float64
Variaciones_Tasa         float64
Variaciones_Cuad_Tasa    float64
dtype: object

Habiendo verificado que nuestro DataFrame es correcto, obtenemos la media y las desviaciones
estándar de nuestros datos:

In [31]:
media_variacion = datos_LALA18_variaciones['Variaciones_Tasa'].mean()
media_variacion_cuad = datos_LALA18_variaciones['Variaciones_Cuad_Tasa'].mean()

stdev_variacion = datos_LALA18_variaciones['Variaciones_Tasa'].std()
stdev_variacion_cuad = datos_LALA18_variaciones['Variaciones_Cuad_Tasa'].std()

Procedemos entonces a la valuación

In [41]:
### INICIA FUNCIÓN ###
def tasa_mercado(datos):
    """
    Función para obtener la Tasa de Mercado, obtiene el último valor de la tasa
    que se encuentra en el DataFrame
    :param datos: DataFrame con tasas
    :return: Úlimo valor en la columna de tasas
    """
    resp = (datos.iloc[:,1].iloc[-1]) / 100     ### Segunda columna, último valor
    return resp
### FIN DE FUNCIÓN ###

tasaMercado = tasa_mercado(datos_LALA18_variaciones)


Creamos una función que traiga a valor presente los flujos:

In [47]:
def flujos_tmas1_hoy(cuponesCompletos, tasaMkt, periodo, valCupon, diasxDevengar):
    """
    Función que obtiene DataFrame de [cupon | flujos t+1 | flujos hoy]
    
    :param cuponesCompletos: Número de cupones completos
    :param tasaMkt: Tasa de Mercado (último valor de tasa del DATAfRAME de origen
    :param periodo: Periodicidad del cupón
    :param valCupon: Valor del Cupón
    :param diasxDevengar: Días que faltan por devengar
    :return: Dataframe con las 3 columnas
    """
    cupon = np.arange(1, (cuponesCompletos+1))     ### Vector con cupones enteros
    flujosTmas1 = np.zeros(len(cupon))                   ### Vector con flujos calculados
    flujosHoy = np.zeros(len(cupon))                     ### Vector con flujos al día de hoy
    
    vAjustada = 1 / (1 + (tasaMkt * periodo / 360))      ### v de Matemáticas Financieras
    
    ### Llenamos vector de flujos a valor presente en (t+1) y flujos para HOY
    for i in range(len(flujosTmas1)-1):
        flujosTmas1[i] = valCupon * (vAjustada**(cupon[i]))
        flujosHoy[i] = flujosTmas1[i] * (vAjustada**(diasxDevengar / periodo))
        
    ### El último cupón también paga el Valor Nominal del Bono    
    flujosTmas1[len(flujosTmas1)-1] = (VN + valCupon) * (vAjustada**(cupon[len(cupon)-1]))
    flujosHoy[len(flujosHoy)-1] = (flujosTmas1[len(flujosTmas1)-1]) * (vAjustada**(diasxDevengar / periodo))
    
    ### Formamos DataFrame con esta información
    dataFrame1 = pd.DataFrame()     ### DataFrame vacío
    cuponPd = pd.Series(cupon)
    flujosTmas1Pd = pd.Series(flujosTmas1)
    flujosHoyPd = pd.Series(flujosHoy)
    
    dataFrame1['Cupon'] = cuponPd
    dataFrame1['Flujos_t+1'] = flujosTmas1Pd
    dataFrame1['Flujos_hoy'] = flujosHoyPd
    
    return dataFrame1
### FIN  DE LA FUNCIÓN ###

valuacionFlujos = flujos_tmas1_hoy(numeroCuponesCompletos, tasaMercado, periodicidad, valorCupon, diasPorDevengar)
    

El **Precio del Bono** será la suma de los flujos traidos a valor presente a *t+1*

In [49]:
### INICIA FUNCIÓN ###
def precio_bono(datos):
    """
    Función que obtiene el precio del bono como la suma de los flujos en t+1
    La columna con los flujos en t+1 deberá llamarse "Flujos_t+1"
    
    :param datos: DataFrame con columna de flujos en t+1
    :return: Precio del Bono (float)
    """
    resp = datos['Flujos_t+1'].sum()
    return resp

precioBono = precio_bono(valuacionFlujos)

precioBono

99.21109327598595

Entonces, bajo esta técnica de valuación, el precio de nuestro bono es **99.211093**

Existe un "cupón no completo", que deberá traerse a valor presente considerando el 
tiempo restante:

In [50]:
### INICIA FUNCIÓN ###
def cupon_no_completo(valCupon, tasaMkt, periodo, diasxDevengar):
    """
    Obtiene el valor presente del cupón no completo 
    
    :param valCupon: Valor del Cupón
    :param tasaMkt: Tasa de Mercado
    :param periodo: Periodicidad del Bono
    :param diasxDevengar: Días que faltan por devengar
    :return: Valor presente del cupón no completo (float)
    """
    vAjustada = 1 / (1 + tasaMkt * periodo / 360)
    resp = valCupon * (vAjustada**(diasxDevengar / periodo))
    return resp

vpCuponNoCompleto = cupon_no_completo(valorCupon, tasaMercado, periodicidad, diasPorDevengar)

vpCuponNoCompleto
    
    



4.4713871526860745